In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
res = requests.get('https://automatetheboringstuff.com/files/rj.txt')

In [6]:
with open('ch11_project/RomeoAndJuliet.txt', 'wb') as f:
    for chunk in res.iter_content(10000):
        f.write(chunk)

In [21]:
url = 'https://forecast.weather.gov/MapClick.php?lat=37.78862000000004&lon=-122.39508999999998'
res = requests.get(url)
soup = BeautifulSoup(res.text)
tem = soup.select('p.myforecast-current-lrg')

In [26]:
tem[0].get_text()

'57°F'

In [27]:
tem[0].attrs

{'class': ['myforecast-current-lrg']}

In [42]:
url = 'https://weibo.cn'

In [45]:
headers = {'cookie': 'SCF=AjRCQ53oHqFjbwn3NUPNfX1cO5WzK1kIjjW9T5JvKZv53yL4z3hY37D38ucaeLuPFgFD-Yb40LZvvKKISOWnpcQ.; ALF=1546784588; _T_WM=9c8e14ef5e9bbee194b5d95a597c9351; SUB=_2A25xDvxeDeRhGeBM7VAR-CnOzT6IHXVS8IQWrDV6PUJbkdAKLULwkW1NRNqh1R1Bw8aynqv2OzgsLOc6-iDEw5Dq; SUHB=0FQ7mXH3ZgoRzl; SSOLoginState=1544195086',
           'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text)

In [50]:
soup.select('#M_H7ABsq2Ap > div:nth-of-type(1) > span.ctt')

[<span class="ctt">全场结束！！梅西帽子戏法+两个助攻，苏亚雷斯皮克进球，巴萨客场5-0莱万特！ ​​​</span>]

In [51]:
soup.select('#M_H7wB7vWM6 > div:nth-of-type(1) > span')

[<span class="ctt">:三天没发微博 今夜里又有巴萨比赛 为了不让前两条说球的微博接着又是一条说球的微博 我决定发了这条既不成熟又没意义的微博... ​​​</span>]

# Start

In [19]:
url = 'https://weibo.cn/u/1810527104'

In [4]:
headers = {
    'cookie': '_T_WM=fc88b5b4110bb85436422b37ed874566; SUB=_2A25xE2kJDeRhGedJ7FUU8S3PwzmIHXVS_HdBrDV6PUJbkdAKLVX7kW1NUao5oVXR5SZ9sufxvnS2oOeH0lZyHEPl; SUHB=0XDUWdi2Jn_NxS; SCF=AnccQdTUMv0UCPDWHMLGwFy4Y3r6mCzDXVrMtT_lbjUi8K5g8RTe6fNTMPPQkc3hkC3kFZjfk0KhUCj1dPbbHn8.; SSOLoginState=1545017689',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}

wb_data = requests.get(url, headers=headers)
soup = BeautifulSoup(wb_data.text)

In [5]:
# #pagelist > form > div > input[type="hidden"]:nth-child(2)
soup.select('#pagelist > form > div > input[type="hidden"]')[0].get('value')

'92'

In [6]:
def get_total_pages():
    total = int(soup.select('#pagelist > form > div > input[type="hidden"]')[0].get('value'))
    return ['https://weibo.cn/u/1810527104?page={}'.format(i) for i in range(1, total + 1)]

get_total_pages()[:3]

['https://weibo.cn/u/1810527104?page=1',
 'https://weibo.cn/u/1810527104?page=2',
 'https://weibo.cn/u/1810527104?page=3']

In [7]:
pieces = soup.find_all('div', {'class': 'c'})

In [8]:
contents = [x for x in pieces if x.has_attr('id')]

In [9]:
piece = contents[0]

In [10]:
piece

<div class="c" id="M_H7zUwDp7p"><div><span class="ctt">昨天在国内参与的第一个群展开幕了www发博以作纪念～ ​​​</span> <span class="cmt">已赞[1]</span> <a href="https://weibo.cn/repost/H7zUwDp7p?uid=1810527104&amp;rl=0">转发[0]</a> <a class="cc" href="https://weibo.cn/comment/H7zUwDp7p?uid=1810527104&amp;rl=0#cmtfrm">评论[0]</a> <a href="https://weibo.cn/fav/addFav/H7zUwDp7p?rl=0&amp;st=3a08c0">收藏</a><!-- --> <span class="ct">今天 03:53 来自iPhone</span></div></div>

In [11]:
pid = piece.get('id')
text = piece.select('span.ctt')[0].text
like = piece.select('span.cmt')[0].text[3:-1]
comment_url = piece.select('a.cc')[0].get('href')


print(pid, text, like, comment_url, sep='\n')

M_H7zUwDp7p
昨天在国内参与的第一个群展开幕了www发博以作纪念～ ​​​
1
https://weibo.cn/comment/H7zUwDp7p?uid=1810527104&rl=0#cmtfrm


In [20]:
a_url = soup.select('div > a.cc')[0].get('href')

In [108]:
import hashlib
import time
import random
import re
import logging

logging.basicConfig(
    filename='trace_new.log',
    level=logging.INFO,
    format='%(asctime)s : %(levelname)s : %(message)s'
)

logging.info('Start tracing new.')

url = 'https://weibo.cn/u/1810527104'

headers = {
    'cookie': '_T_WM=fc88b5b4110bb85436422b37ed874566; SUB=_2A25xE2kJDeRhGedJ7FUU8S3PwzmIHXVS_HdBrDV6PUJbkdAKLVX7kW1NUao5oVXR5SZ9sufxvnS2oOeH0lZyHEPl; SUHB=0XDUWdi2Jn_NxS; SCF=AnccQdTUMv0UCPDWHMLGwFy4Y3r6mCzDXVrMtT_lbjUi8K5g8RTe6fNTMPPQkc3hkC3kFZjfk0KhUCj1dPbbHn8.; SSOLoginState=1545017689',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}

wb_data = requests.get(url, headers=headers)
soup = BeautifulSoup(wb_data.text, 'lxml')
first = soup.select('div > a.cc')[0].get('href')
logging.info('Latest one: {}.'.format(first))
pid = re.compile(r'([A-Za-z0-9]+)\?')


def crawl(url):
    page = requests.get(url, headers=headers)
    filename = pid.findall(url)[0] + '.html'
    logging.info('Create page {} for {}.'.format(filename, url))
    with open(filename, 'wb') as f:
        for chunk in page.iter_content(10000):
            f.write(chunk)
            

while True:
    time.sleep(random.randint(300, 900))
    wb_data = requests.get(url, headers=headers)
    soup = BeautifulSoup(wb_data.text, 'lxml')
    current_first = soup.select('div > a.cc')[0].get('href')
    if current_first != first:
        logging.info('Find new post\'s {}.'.format(current_first))
        crawl(current_first)
        first = current_first

In [75]:
contents = requests.get(a_url, headers=headers)

In [22]:
contents_soup = BeautifulSoup(contents.text, 'lxml')

In [23]:
contents_soup

<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="no-cache" http-equiv="Cache-Control"/><meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0, maximum-scale=2.0" id="viewport" name="viewport"/><link color="black" href="https://h5.sinaimg.cn/upload/2015/05/15/28/WeiboLogoCh.svg" mask="" rel="icon" sizes="any"/><meta content="240" name="MobileOptimized"/><title>评论列表</title><style id="internalStyle" type="text/css">html,body,p,form,div,table,textarea,input,span,select{font-size:12px;word-wrap:break-word;}body{background:#F8F9F9;color:#000;padding:1px;margin:1px;}table,tr,td{border-width:0px;margin:0px;padding:0px;}form{margin:0px;padding:0px;border:0px;}textarea{border:1px solid #96c1e6}textarea{width:95%;}a,.tl{color:#2a5492;text-decoration:under

In [70]:
po = (
    contents_soup.select_one('#M_ > div > a').text +
    contents_soup.select_one('#M_ > div > span.ctt').text.replace('\u200b', ' ') +
    contents_soup.select_one('#M_ > div > span.ct').text.strip()
)

comment = (
    contents_soup.select_one('#C_4318063493299828 > a').text + ":" +
    contents_soup.select_one('#C_4318063493299828 > span.ctt').text + "\n" +
    contents_soup.select_one("#C_4317790909499989 > a").text + ":" +
    contents_soup.select_one("#C_4317790909499989 > span.ctt").text
)

In [71]:
print(info)
print(comment)

一条读弗洛伊德的鱼:买书…买书使人贫穷……[允悲]    12月16日 04:53
一条读弗洛伊德的鱼:回复@许山山丶:非常押韵了！
许山山丶:但读书使鱼富有！


In [105]:
s = 'https://weibo.cn/comment/H7zUwDp7p?uid=1810527104&rl=0#cmtfrm'
p = re.compile(r'([A-Za-z0-9]+)\?')
p.findall(s)[0]

'H7zUwDp7p'

In [109]:
crawl('https://weibo.cn/comment/H3XY0s4XG?ckAll=1')

In [112]:
import logging

In [111]:
loger = logging.getLogger()

In [ ]:
loger.

In [ ]:
# All the code
url = 'https://weibo.cn/u/1810527104'

headers = {
    'cookie': '_T_WM=fc88b5b4110bb85436422b37ed874566; SUB=_2A25xE2kJDeRhGedJ7FUU8S3PwzmIHXVS_HdBrDV6PUJbkdAKLVX7kW1NUao5oVXR5SZ9sufxvnS2oOeH0lZyHEPl; SUHB=0XDUWdi2Jn_NxS; SCF=AnccQdTUMv0UCPDWHMLGwFy4Y3r6mCzDXVrMtT_lbjUi8K5g8RTe6fNTMPPQkc3hkC3kFZjfk0KhUCj1dPbbHn8.; SSOLoginState=1545017689',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}

wb_data = requests.get(url, headers=headers)
soup = BeautifulSoup(wb_data.text)

def get_total_pages():
    total = int(soup.select('#pagelist > form > div > input[type="hidden"]')[0].get('value'))
    return ['https://weibo.cn/u/1810527104?page={}'.format(i) for i in range(1, total + 1)]


def start():
    pages = get_total_pages()
    for page in pages:
        crawl(page)